# Arbol de decision con _CART_ (Clasificación).

Para la generación de un árbol de decisión utilizando el algoritmo CART _(Classification and Regression Trees)_, se necesita trabajar con un **DataFrame**, el cual puede ser manipulado eficientemente mediante la librería Pandas. Esta librería será de gran ayuda para organizar, limpiar y transformar los datos antes de construir el modelo.

### Instalación de librerías.

Para instalar la librería pandas, utilizaremos pip. Antes de hacerlo, es recomendable actualizar pip con el siguiente comando:
 ``` Bash
 python -m pip install --upgrade pip
 ```
Una vez actualizado, podemos instalar Pandas con el siguiente comando: 
``` Bash
pip install pandas
```

Además de pandas, utilizaremos la librería NumPy. Por defecto, NumPy se instala automáticamente junto con pandas, ya que es una de sus dependencias. Sin embargo, si por alguna razón no se instala correctamente, puedes instalarla de forma manual con el siguiente comando:
```Bash
pip install numpy
```
Una vez importadas las librerías necesarias, procederemos a cargar nuestro conjunto de datos en un DataFrame utilizando la librería pandas.

In [43]:
import pandas as pd
import numpy as np

df = pd.read_csv('Data/DataFrame_Tennis.csv')
df

,Clima,Temperatura,Humedad,Viento,Salir
0,soleado,alta,alta,debil,no
1,soleado,alta,alta,fuerte,no
2,nublado,alta,alta,debil,si
3,lluvioso,media,alta,debil,si
4,lluvioso,baja,normal,debil,si
5,lluvioso,baja,normal,fuerte,no
6,nublado,baja,normal,fuerte,si
7,soleado,media,alta,debil,no
8,soleado,baja,normal,debil,si
9,lluvioso,media,normal,debil,si


Es importante tener en cuenta que el algoritmo CART funciona de manera más eficiente cuando los datos son numéricos. El uso de variables categóricas puede dificultar la interpretación del modelo y aumentar su complejidad, ya que CART divide el espacio de decisiones mediante comparaciones binarias.

Por esta razón, es recomendable convertir las variables categóricas a valores numéricos. Una forma práctica de hacerlo en pandas es utilizando el atributo `.cat.codes`, el cual está disponible para columnas con tipo de dato `category`.


In [44]:

for col in df.select_dtypes(include=['object']):
    df[col] = df[col].astype('category').cat.codes

df

,Clima,Temperatura,Humedad,Viento,Salir
0,2,0,0,0,0
1,2,0,0,1,0
2,1,0,0,0,1
3,0,2,0,0,1
4,0,1,1,0,1
5,0,1,1,1,0
6,1,1,1,1,1
7,2,2,0,0,0
8,2,1,1,0,1
9,0,2,1,0,1


### Arbol binario
La clase `Nodo` define la estructura básica de un árbol de decisión. Cada nodo puede representar una división del conjunto de datos utilizando un **atributo** (una columna del DataFrame) y un **umbral** (valor de corte para esa división). Si el nodo es una **hoja**, se le asigna una **clase**, que indica la categoría final del modelo. Además, contiene dos enlaces (`izq` y `der`) que apuntan a los subárboles izquierdo y derecho, respectivamente, lo que permite construir el árbol de forma recursiva.


In [45]:
class Nodo:
    def __init__(self, atributo=None, umbral=None, clase=None):
        self.atributo = atributo      
        self.umbral = umbral         
        self.clase = clase           
        self.izq = None             
        self.der = None             

### Umbral

El **Umbral** o _Threshold_ $(t)$  es un valor numérico que se utiliza para dividir un conjunto de datos en dos partes dentro de un nodo del árbol de decisión. Para encontrar el umbral óptimo, se deben ordenar los elementos del conjunto y calcular el promedio entre cada par de valores consecutivos, de la siguiente manera:

$$
T = \left\{ \frac{v_i + v_{i+1}}{2} \;\middle|\; i = 1, 2, \dots, n-1 \right\}
$$


In [46]:
def calcular_umbral(df, col_idx):
    valores = np.sort(df.iloc[:, col_idx].unique())
    return [(valores[i] + valores[i+1]) / 2 for i in range(len(valores)-1)]

print(calcular_umbral(df,0))

[np.float64(0.5), np.float64(1.5)]


### Indice Gini

El criterio de división utilizado en los árboles de clasificación es el índice de Gini, este mide la impureza de un nodo después de la división.

$$
Gini = 1 - \sum_{i = 0} p_i^2
$$

In [47]:
def gini(probabilidades):
    return 1 - sum(p ** 2 for p in probabilidades)

### Split

Para seleccionar el mejor atributo y punto de corte, se calcula el índice de Gini para cada nodo hijo generado por la división, y luego se realiza una suma ponderada de estos valores según la proporción de instancias en cada nodo (probabilidad relativa). La división que minimiza esta suma ponderada se selecciona como la mejor.

$$
Gini_{\text{split}} = \frac{|D_{\text{izq}}|}{|D|} \cdot Gini(D_{\text{izq}}) + \frac{|D_{\text{der}}|}{|D|} \cdot Gini(D_{\text{der}})
$$

In [48]:
def encontrar_mejor_split(df):
    mejor_gini = float('inf')
    mejor_attr = None
    mejor_umbral = None
    n = df.shape[0]   #Devuelve el numero de filas
    for col in range(df.shape[1] - 1):
        for u in calcular_umbral(df, col):
            izq = df[df.iloc[:, col] <= u]
            der = df[df.iloc[:, col] > u]
            if izq.empty or der.empty:
                continue
            p_izq = izq.iloc[:, -1].value_counts(normalize=True).values
            p_der = der.iloc[:, -1].value_counts(normalize=True).values
            gini_total = (len(izq)/n)*gini(p_izq) + (len(der)/n)*gini(p_der)
            if gini_total < mejor_gini:
                mejor_gini = gini_total
                mejor_attr = col
                mejor_umbral = u
    return mejor_attr, mejor_umbral

print(encontrar_mejor_split(df))

(2, np.float64(0.5))


### Constucción del arbol

El árbol de decisión se construye a partir de preguntas recursivas basadas en un atributo y un umbral específico. En cada nodo, se determina qué atributo y valor de corte dividen mejor el conjunto de datos. Luego, se crean nodos hijos de manera recursiva hasta que todas las instancias en un nodo pertenezcan a la misma clase o no sea posible continuar dividiendo.

In [49]:
def construir_arbol(df):
    clases = df.iloc[:, -1]
    if clases.nunique() == 1:
        return Nodo(clase=clases.iloc[0])
    
    attr, u = encontrar_mejor_split(df)
    if attr is None:
        return Nodo(clase=clases.mode()[0])
    
    nodo = Nodo(atributo=attr, umbral=u)
    izq = df[df.iloc[:, attr] <= u]
    der = df[df.iloc[:, attr] > u]
    nodo.izq = construir_arbol(izq)
    nodo.der = construir_arbol(der)
    return nodo


### Visualización del árbol



In [52]:
def imprimir_arbol(nodo, nivel=0):
    if nodo is None:
        return
    sangria = "  " * nivel
    if nodo.clase is not None:
        print(f"{sangria}Hoja: clase = {nodo.clase}")
    else:
        print(f"{sangria}[{df.iloc[:, :-1].columns[nodo.atributo]} ≤ {nodo.umbral}]")
        imprimir_arbol(nodo.izq, nivel+1)
        imprimir_arbol(nodo.der, nivel+1)

arbol = construir_arbol(df)
print("Estructura del árbol CART:")
imprimir_arbol(arbol)

Estructura del árbol CART:
[Humedad ≤ 0.5]
  [Clima ≤ 1.5]
    [Clima ≤ 0.5]
      [Viento ≤ 0.5]
        Hoja: clase = 1
        Hoja: clase = 0
      Hoja: clase = 1
    Hoja: clase = 0
  [Clima ≤ 0.5]
    [Viento ≤ 0.5]
      Hoja: clase = 1
      Hoja: clase = 0
    Hoja: clase = 1
